# TEST

[new text]{.SA}~~old text~~

[new text]{.MK}~~old text~~

[new text]{.MP}~~old text~~

[new text]{.JK}~~old text~~

[new text]{.JL}~~old text~~

[new text]{.TO}~~old text~~

[new text]{.SJ}~~old text~~

[new text]{.MC}~~old text~~

# Introduction 
Mixture of Experts (MoE) architectures have recently garnerned considerable attention for their ability to increase the size of Transformer architectures while keeping the computational cost of training and inference constant. The most successful MoE approaches achieve this by activating only subsets of a very large feed forward layer for each processed token (the alternative subsets of parameters are often called _experts_). 
[Komentarz: Czy są MoE które nie wybierają tylko części parametrów FF dla danego tokena? ]{.MK}[Most likely there are, but we are concerned with the most successful approaches]{.SA}

This technique comes at a cost, though: the operation of choosing the most suitable experts for a given token is discrete, and as such learning how to choose is difficult; the models are known to suffer from issues such as training instability, expert under- and overload and more. While some of those problems can be alleviated with e.g. the use of various auxiliary losses or reduced initialisation scale, it is certain that existing MoE techniques are more difficult and less intuitive to train. 

With this motivation, we propose Mixture of Tokens: a new, fully-differentiable type of architecture that retains all efficiency benefits of MoE and alleviates the aformentioned problems by _mixing tokens_ before feeding them into the FeedForward layer instead of routing tokens to experts, in effect allowing all experts to learn from all tokens. This technique is also fully compatible with both masked and causual LLM training.

# Motivation 
### Scaling Language Models
Numerous works [@kaplan2020scaling; @hoffmann2022training] have studied benefits of increasing the size of language models for their performance. However, this scaling comes at a very high computational cost [@strubell2019energy]. Therefore, efficient use of computational resources and network parameters is often recognized as one of the most important challenges in the area [@rae2022scaling; @jaszczur2021sparse; @nawrot2022hierarchical].

### Mixture of Experts
The core idea in Mixture of Experts is to increase model size without incurring additional computational costs. Roughly speaking, is is done by replacing the FeedForward layer standard for Transformer architectures with a set of experts. In this new setup, a given token is processed only by the most suitable expert(s), decided by a small routing network.



```{css, echo=FALSE, eval=TRUE}
.multiple .column {
  width: 15%;
  margin: 5px;
}

.multiple .columns {
  display: flex;
}

@media (max-width: 767px) {
  .multiple .column {
    width: auto;
  }

 .multiple .columns {
   display: flex;
   flex-direction: column;
  }
}
```




[this is old]{.SA}
The increased model size, despite keeping computational cost during training and inference constant, leads to significant performance improvements, and has been shown to behave predictably with respect to parameter count and inference compute [scaling moe], similarly to the standard dense Transformer models [scaling,chinchilla]. 
However, following the success of those initial huge parameter-count models, thorough analysis of the model training and inference schemes uncovered that using Mixture of Experts techniques brings about an entirely new suite of challenges.


# Background
In recent years, there has been a growing interest in methods of increasing the number of parameters in neural networks while maintaining their computational demands. A major advance in eliminating some problems with traditional MoE approaches is Expert Choice (here biblio). Our work has been inspired by their approach - MoT can be seen as an Expert Choice (as opposed to *Token Choice*) kind of method.

There is also concurrent work (Softmoe) investigating ideas similar to ours in the vision domain. It is, however, limited to encoder-only models. It is worth noting that earlier, other fully-differentiable MoE approaches had been proposed (biblio copied from Softmoe). /* jakoś inaczej sformułowac: However, expert-mixing strategies presented in those works incur significant computational overhead. Also, they are token choice instead of expert choice, so different than us. */

## Limitations of Current Approaches (sekcja 8.2 w paperze) SA 
<!-- JK - byłoby bardziej logicze dać tę sekcję pod related work -->
Without mentioning papers 
- obecne podejścia są niestabilne
- mamy idealny balancing token -> expert

with training stability, management of expert _load_ (how many tokens are routed to each expert) that lead to time overheads, mode collapse (all tokens routed to a small portion of the expert set), leaving some tokens unprocessed, problems with expert exploration and specialization/redundancy, low performance of discrete operations on accelerators and finally communication costs in parallel training.~~
[...old text]

# Background [MK]

```{=html}
<html>
<head>
<title>generic_moe</title>
<meta charset="utf-8"/>
</head>
<body><div class="mxgraph" style="max-width:100%;border:1px solid transparent;" data-mxgraph="{&quot;highlight&quot;:&quot;#0000ff&quot;,&quot;nav&quot;:true,&quot;resize&quot;:true,&quot;toolbar&quot;:&quot;zoom layers tags lightbox&quot;,&quot;edit&quot;:&quot;_blank&quot;,&quot;xml&quot;:&quot;&lt;mxfile host=\&quot;app.diagrams.net\&quot; modified=\&quot;2023-10-02T16:14:50.668Z\&quot; agent=\&quot;Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0\&quot; etag=\&quot;bNLqouWxuNlX6NyIyFeK\&quot; version=\&quot;21.6.5\&quot; type=\&quot;google\&quot;&gt;\n  &lt;diagram name=\&quot;Page-1\&quot; id=\&quot;Ooj4VCty3gAyGvrI8WI3\&quot;&gt;\n    &lt;mxGraphModel dx=\&quot;1602\&quot; dy=\&quot;2964\&quot; grid=\&quot;1\&quot; gridSize=\&quot;10\&quot; guides=\&quot;1\&quot; tooltips=\&quot;1\&quot; connect=\&quot;1\&quot; arrows=\&quot;1\&quot; fold=\&quot;1\&quot; page=\&quot;1\&quot; pageScale=\&quot;1\&quot; pageWidth=\&quot;827\&quot; pageHeight=\&quot;1169\&quot; math=\&quot;0\&quot; shadow=\&quot;0\&quot;&gt;\n      &lt;root&gt;\n        &lt;mxCell id=\&quot;0\&quot; /&gt;\n        &lt;mxCell id=\&quot;1\&quot; parent=\&quot;0\&quot; /&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-1\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot; style=&amp;quot;font-size: 11px;&amp;quot;&amp;gt;ATTENTION&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#f8cecc;strokeColor=#592927;fontSize=11;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;39\&quot; y=\&quot;-932\&quot; width=\&quot;81\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-2\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;html=1;rounded=0;strokeWidth=1;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-822\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-1152\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-6\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;40\&quot; y=\&quot;-1082\&quot; width=\&quot;80\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-9\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;fontFamily=Helvetica;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;371\&quot; y=\&quot;-1010\&quot; width=\&quot;79\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-24\&quot; value=\&quot;\&quot; style=\&quot;endArrow=classic;html=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;startSize=3;endSize=3;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-9\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;410\&quot; y=\&quot;-930\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;410\&quot; y=\&quot;-960\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-25\&quot; value=\&quot;\&quot; style=\&quot;endArrow=classic;html=1;rounded=0;startSize=3;endSize=3;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;410\&quot; y=\&quot;-1010\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;410\&quot; y=\&quot;-1050\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot; style=&amp;quot;font-size: 11px;&amp;quot;&amp;gt;controller&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=0;whiteSpace=wrap;html=1;fillColor=#ffe6cc;strokeColor=#d79b00;fontSize=11;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;670\&quot; y=\&quot;-880\&quot; width=\&quot;60\&quot; height=\&quot;20\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-27\&quot; value=\&quot;\&quot; style=\&quot;endArrow=classic;html=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;endSize=3;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;700\&quot; y=\&quot;-820\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;310\&quot; y=\&quot;-1040\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-33\&quot; value=\&quot;\&quot; style=\&quot;endArrow=blockThin;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;endSize=4;endFill=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;710\&quot; y=\&quot;-870\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;839.5\&quot; y=\&quot;-970\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-34\&quot; value=\&quot;\&quot; style=\&quot;endArrow=blockThin;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;endSize=4;endFill=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;710\&quot; y=\&quot;-870\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;749.5\&quot; y=\&quot;-970\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-35\&quot; value=\&quot;\&quot; style=\&quot;endArrow=blockThin;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;endSize=4;endFill=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;700\&quot; y=\&quot;-880\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;659.5\&quot; y=\&quot;-970\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-38\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN1&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;fontFamily=Helvetica;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;531\&quot; y=\&quot;-1010\&quot; width=\&quot;79\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-39\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN2&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;fontFamily=Helvetica;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;621\&quot; y=\&quot;-1010\&quot; width=\&quot;79\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-40\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN3&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;fontFamily=Helvetica;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;711\&quot; y=\&quot;-1010\&quot; width=\&quot;79\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-41\&quot; value=\&quot;&amp;lt;font color=&amp;quot;#363636&amp;quot;&amp;gt;FFN4&amp;lt;/font&amp;gt;\&quot; style=\&quot;rounded=1;whiteSpace=wrap;html=1;fillColor=#dae8fc;strokeColor=#000066;fontFamily=Helvetica;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;801\&quot; y=\&quot;-1010\&quot; width=\&quot;79\&quot; height=\&quot;40\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-45\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;endSize=0;endFill=0;exitX=0.456;exitY=-0.079;exitDx=0;exitDy=0;exitPerimeter=0;entryX=1;entryY=1;entryDx=0;entryDy=0;startSize=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-41\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;761\&quot; y=\&quot;-1000\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;760\&quot; y=\&quot;-1090\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-48\&quot; value=\&quot;\&quot; style=\&quot;endArrow=blockThin;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;endSize=4;endFill=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-26\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-38\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;710\&quot; y=\&quot;-870\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;669.5\&quot; y=\&quot;-960\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; value=\&quot;\&quot; style=\&quot;ellipse;whiteSpace=wrap;html=1;aspect=fixed;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; vertex=\&quot;1\&quot;&gt;\n          &lt;mxGeometry x=\&quot;703\&quot; y=\&quot;-1101\&quot; width=\&quot;3\&quot; height=\&quot;3\&quot; as=\&quot;geometry\&quot; /&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-50\&quot; value=\&quot;\&quot; style=\&quot;endArrow=classic;html=1;rounded=0;startSize=3;endSize=3;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;709.9999999999998\&quot; y=\&quot;-1130\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;705\&quot; y=\&quot;-1140\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-51\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;endSize=0;endFill=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-40\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;847\&quot; y=\&quot;-1003\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;738\&quot; y=\&quot;-1102\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-52\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;endSize=0;endFill=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-39\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;761\&quot; y=\&quot;-1000\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;724\&quot; y=\&quot;-1093\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-53\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;dashPattern=1 3;strokeWidth=1;rounded=0;endSize=0;endFill=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-38\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-49\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;671\&quot; y=\&quot;-1000\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;698\&quot; y=\&quot;-1092\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-57\&quot; value=\&quot;\&quot; style=\&quot;curved=1;endArrow=classic;html=1;rounded=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;startSize=10;endSize=3;\&quot; parent=\&quot;1\&quot; source=\&quot;FTCFTmSgrVzz-DVGMluv-6\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;120\&quot; y=\&quot;-1082\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-1112\&quot; as=\&quot;targetPoint\&quot; /&gt;\n            &lt;Array as=\&quot;points\&quot;&gt;\n              &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-1112\&quot; /&gt;\n              &lt;mxPoint x=\&quot;30\&quot; y=\&quot;-1112\&quot; /&gt;\n            &lt;/Array&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-59\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;rounded=0;strokeColor=#4D4D4D;dashPattern=1 2;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-1152\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-1172\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-60\&quot; value=\&quot;\&quot; style=\&quot;endArrow=none;dashed=1;html=1;rounded=0;strokeColor=#4D4D4D;dashPattern=1 2;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-802\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-822\&quot; as=\&quot;targetPoint\&quot; /&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-68\&quot; value=\&quot;\&quot; style=\&quot;curved=1;endArrow=classic;html=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;strokeColor=#4D4D4D;endSize=3;\&quot; parent=\&quot;1\&quot; target=\&quot;FTCFTmSgrVzz-DVGMluv-6\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-992\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;110\&quot; y=\&quot;-1042\&quot; as=\&quot;targetPoint\&quot; /&gt;\n            &lt;Array as=\&quot;points\&quot;&gt;\n              &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-1012\&quot; /&gt;\n              &lt;mxPoint x=\&quot;40\&quot; y=\&quot;-1012\&quot; /&gt;\n              &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-1012\&quot; /&gt;\n            &lt;/Array&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-69\&quot; value=\&quot;\&quot; style=\&quot;curved=1;endArrow=classic;html=1;rounded=0;entryX=0.5;entryY=1;entryDx=0;entryDy=0;strokeColor=#4D4D4D;endSize=3;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-842\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-892\&quot; as=\&quot;targetPoint\&quot; /&gt;\n            &lt;Array as=\&quot;points\&quot;&gt;\n              &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-862\&quot; /&gt;\n              &lt;mxPoint x=\&quot;40\&quot; y=\&quot;-862\&quot; /&gt;\n              &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-862\&quot; /&gt;\n            &lt;/Array&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n        &lt;mxCell id=\&quot;FTCFTmSgrVzz-DVGMluv-70\&quot; value=\&quot;\&quot; style=\&quot;curved=1;endArrow=classic;html=1;rounded=0;exitX=0.5;exitY=0;exitDx=0;exitDy=0;strokeColor=#4D4D4D;startSize=10;endSize=3;\&quot; parent=\&quot;1\&quot; edge=\&quot;1\&quot;&gt;\n          &lt;mxGeometry width=\&quot;50\&quot; height=\&quot;50\&quot; relative=\&quot;1\&quot; as=\&quot;geometry\&quot;&gt;\n            &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-932\&quot; as=\&quot;sourcePoint\&quot; /&gt;\n            &lt;mxPoint x=\&quot;10\&quot; y=\&quot;-962\&quot; as=\&quot;targetPoint\&quot; /&gt;\n            &lt;Array as=\&quot;points\&quot;&gt;\n              &lt;mxPoint x=\&quot;80\&quot; y=\&quot;-962\&quot; /&gt;\n              &lt;mxPoint x=\&quot;30\&quot; y=\&quot;-962\&quot; /&gt;\n            &lt;/Array&gt;\n          &lt;/mxGeometry&gt;\n        &lt;/mxCell&gt;\n      &lt;/root&gt;\n    &lt;/mxGraphModel&gt;\n  &lt;/diagram&gt;\n&lt;/mxfile&gt;\n&quot;}"></div>
<script type="text/javascript" src="https://viewer.diagrams.net/js/viewer-static.min.js"></script>
</body>
</html>
```





# Method [SJ]

[Link to diagrams: https://drive.google.com/file/d/1jvew0ft43zpaG-iCk7Ob2ilJw7VjsyeZ/view?usp=sharing ]{.SJ}

[Move this paragraph to background]{.SJ}
The general idea of MoT is to run multiple experts which independently aggregate and process tokens. The most common (and original) form of MoE, token-choice, operates from the perspective of a single token. [This paragraph will be more detailed in background section.]{.SJ} However, MoT is more similar to expert-choice, where it is an expert who decides which tokens to process.

[comparison of both token-choice and expert-choice]{.SJ}


```{=html}
<iframe frameborder="0" style="width:100%;height:512px;" src="https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&page-id=77xPu9R-wu4rnPo6aUtL&title=Untitled%20Diagram.drawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1jvew0ft43zpaG-iCk7Ob2ilJw7VjsyeZ%26export%3Ddownload"></iframe>
```




In MoT, each expert works independently from one another. [The only dependence is during experts' outputs aggregation, but it's a matter of rescaling each output before adding it to the residual, so we will skip it/explain it in later section.]{.SJ} Therefore, to explain the inner workings of MoT we will focus on a single expert. See the diagram below.


```{=html}
<iframe width="780" height="500" src="https://viewer.diagrams.net/index.html?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&page-id=abJAIyk8VqZmFR_1QRqh&title=Untitled%20Diagram.drawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1jvew0ft43zpaG-iCk7Ob2ilJw7VjsyeZ%26export%3Ddownload#%7B%22pageId%22%3A%22abJAIyk8VqZmFR_1QRqh%22%7D
" title="Figure"></iframe>
```



This single expert works as a normal feedforward layer[(maybe a diagram of lin1->relu->lin2?)]{.SJ}, except for a merger and an emitter placed to process the input and the output, respectively. Each merger takes a group of tokens and computes a weighted average of them (according to weights described in later section). This is similart in spirit to the controller in expert-choice MoE - controller took a group of tokens and chose and passed only one of them (or top-k of them in general, but for now let's assume k=1). Similarly, the emitter which processes the output of the expert layer, just scales the output of the layer by a respective token's weight, and passes the resulting vector as the update for this token. Note this is, again, similart to expert choice - you can just image the weight being a one-hot encoding. [Maybe another diagram showing one-hot encoding vs MoT]{.SJ}

Now we get to the part of computing weights. The weights are the same [(are they really?)]{.SJ} for both merger and emitter.


```{=html}
<iframe frameborder="0" style="width:100%;height:593px;" src="https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&page-id=C2AunCsj_dl_T9ErqCql&title=Untitled%20Diagram.drawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1jvew0ft43zpaG-iCk7Ob2ilJw7VjsyeZ%26export%3Ddownload"></iframe>
```




How can we apply this technique in decoder? We need a kind of grouping that will not leak information into future tokens. We can achieve that by grouping tokens by position.

```{=html}
<iframe frameborder="0" style="width:100%;height:400px;" src="https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1&page-id=fbuRVmtAR91k7QgiWp5u&title=MoT_diagrams_sj.drawio#Uhttps%3A%2F%2Fdrive.google.com%2Fuc%3Fid%3D1jvew0ft43zpaG-iCk7Ob2ilJw7VjsyeZ%26export%3Ddownload"></iframe>

```




## Implementation
<!-- MK -->
In the first phase of our MoT Layer, we partition batch sequences into token gropus of equal size. 

Then for each group, we calculate a two sets of weights, named _merge_ and _emit_. Each set of weights sum to 1.0, signifying the percentage of each token to utilize.

For every group, using its _merge_ weights, we merge belonging tokens into a single token. Following this, we process all merged tokens with linear experts' weights and a relu activation function. [Komentarz: To zdecydowanie trzeba lepiej napisać]{.MK} 
Employing _emit_ weights, we redistribute resultant token after the experts' layer back to a group of tokens. Lastly, we join the groups to align with the Feed Forward layer input dimensions.

## Encoder vs Decoder
In encoder-only models, each token may attend to the whole input sequence, whereas decoder-only models only allow attending to the tokens that came before a particular token to simulate the autoregressive decoding used during model inference. This is crucial for the training of decoder-only models, as allowing a token to attend to future tokens would constitute a data leak, preventing successful training.
A major problem potentially facing any token-mixing strategy is the possibility of a data leak, making the strategy inappropriate for use with decoder-only models. In MoT, the tokens are mixed across batch, i.e. with other tokens at the same position in other samples in the same training batch. We don't consider other strategies which would be more suitable in the context of encoders.


# Experiments MP / SA
## Experimental setup
In our experiments we train the models for $250{,}000$ steps using a cosine LR scheduler with $2500$ steps of warmup, and final LR equal to $0.1$ of the peak LR. The context length is $256$ tokens and the batch size is $256$.
We tune the max LR separately for each model.

![Performance by step](loss_step.png)
![Performance by step](html-figure.png)

## Co tam dalej
ContMoE vs dense vs expert choice (GPT-BERT mini)
Each run was optimized for that many batches of given size
Contmoe is better than dense


# Upcoming Work
<!-- MK Kto wie czy to dobra nazwa, może Future Work lepsza -->
Coming up in the following weeks, we're ready to take our work up a notch with a few advancements that we're covering in this segment.

## Temperature annealing
In base MoT setup tokens from various samples in the batch are mingled. This theoretically opens doors for being able to tease out the output of one batch sample by another. However, this may pave the way for potential issues.

To tackle this problem, we plan to conduct an experiment where final phase will involve a strategic, gradual reduction of temperature, while simultaneously maintaining a low loss - a process referred to as _temerature annealing_. The anticipated outcome is a discrete-routing model which promotes inference parallelization - a move expected to significantly boost the overall efficiency of our processes.

## Scaling Up Models
In a previous section <TODO check> we showcased results for a BERT-mini FLOP matched MoT. Our preliminary experiments are also suggesting promising outcomes for larger models. In upcoming weeks we aim to prepare comprehensive comparison of larger models with FLOP matched MoT models.


## Do not forget about a BERT
As our results in this post covers only GPT model, in future work we plan to compare results of MoT on BERT model. 


# Conclusions SJ
We have shown the preliminary results showing the promise of MoT improving both the stability of training and final performance of the model. More thorough experiments are underway at the moment and we will release the paper with more results in the coming weeks. In the meantime, you can contact us with any feedback you have at llm.random.team@gmail.com .